# Modelling data

In [5]:
import pandas as pd

datasets = pd.read_pickle( 'datasets.p' )

g_pop = datasets['populations'].sort_values(['Year'],ascending=False)

train_df = datasets['air'].copy(deep=True)
train_df = train_df.reindex(columns = train_df.columns.tolist()
                                  + ['population'])
# display( g_pop.head( ) )

city_dfs = [ ]
for country in train_df[ 'country' ].unique( ):
    print( f'looking for { country }' )
    country_df = train_df[ ( train_df[ 'country' ] == country ) ]
    unique_cities = country_df[ 'city' ].unique( )
    
    for city in unique_cities:
        found_year_data = { }
        
        for year in country_df[ 'year' ].unique( ):
            # try and find year, city and country in g_pop
            population_data = g_pop[ ( g_pop[ 'Year' ] == year ) & ( g_pop[ 'country' ] == country ) & ( g_pop[ 'city' ] == city ) ] 
            
            if len( population_data ) != 0:
                found_year_data[ int( population_data[ 'Year' ].mean( ) ) ] = population_data[ 'Value' ].mean( )
            else:
                found_year_data[ year ] = 0
        
        found_year_data = dict( sorted( found_year_data.items( ) ) )
        
        last_year = list( found_year_data.keys( ) )[ 0 ] 
        last_fill = 0
        first_fill = 0
        
        # This should fill everything inbetween
        years = sorted( list( found_year_data.keys( ) ) )
        for year in years:
            pop = found_year_data[ year ]
            if pop != 0 and last_year != 0:
                if first_fill == 0:
                    first_fill = year
                
                last_fill = year
                
                diff = year - last_year
                if diff == 1:
                    last_year = year
                elif diff > 1:
                    step = int( ( found_year_data[ year ] - found_year_data[ last_year ] ) / diff )
                    for i in range( 1, diff ):
                        found_year_data[ last_year + i ] = found_year_data[ last_year ] + i * step
                continue
        
        avg_growth = 0
        if last_fill - first_fill != 0:
            avg_growth = ( found_year_data[ last_fill ] - found_year_data[ first_fill ] ) / ( last_fill - first_fill )

        
        # filling start
        from_ = list( found_year_data.keys( ) )[ 0 ]
        if from_ == 0:
            to_ = 0
            for year, pop in found_year_data.items( ):
                # finding until when theres data
                if pop != 0:
                    to_ = year
                    break

            for i in range( 1, to_ - from_ + 1 ):
                year_to_fill = to_ - i
                fill_val = found_year_data[ to_ ] - avg_growth * i

                found_year_data[ year_to_fill ] = found_year_data[ to_ ] - avg_growth * i

        # filling end
        start = last_fill + 1
        end = years[ -1 ]
        
        if start != 1: # check if everything isnt 0
            for i in range( end - start ):
                year_to_fill = start + i
                found_year_data[ year_to_fill ] = found_year_data[ start - 1 ] + avg_growth * i
        
        summa = 0
        for year, pop in found_year_data.items( ):
            summa += pop
            
        if summa == 0:
            print( 'No data for', city, country )
            continue
        
        city_df = country_df[ ( country_df[ 'city' ] == city ) ].sort_values( 'year' )
        
        city_df[ 'population' ] = city_df[ 'year' ].map( found_year_data )
        
        city_dfs.append( city_df )

concat_data = pd.concat( city_dfs )

# only where pop isnt 0
train_df = concat_data[ concat_data[ 'population' ] != 0 ]
train_df

looking for Argentina
looking for Australia
looking for Austria
No data for Bratislava Austria
looking for Belgium
No data for Brugge Belgium
No data for Charleroi Belgium
No data for Namur Belgium
looking for Bangladesh
No data for Gazipur Bangladesh
looking for Bulgaria
looking for Bahrain
looking for Bahamas
No data for Nassau Bahamas
looking for Bosnia and Herzegovina
No data for Prijedor Bosnia and Herzegovina
No data for Sarajevo Bosnia and Herzegovina
No data for Tuzla Bosnia and Herzegovina
No data for Zenica Bosnia and Herzegovina
looking for Brazil
looking for Bhutan
No data for Thimphu Bhutan
looking for Canada
No data for Abbotsford Canada
No data for St. Catharines Canada
No data for Sydney Canada
No data for Wellington Canada
looking for Switzerland
looking for Chile
No data for Cerro Navia Chile
No data for El Bosque Chile
No data for Independencia Chile
No data for La Florida Chile
No data for Las Condes Chile
No data for Linares Chile
No data for Pudahuel Chile
No data

No data for Islamabad Pakistan
No data for Lahore Pakistan
No data for Peshawar Pakistan
looking for Peru
No data for Callao Peru
looking for Philippines
looking for Poland
looking for Portugal
No data for Almada Portugal
No data for Braga Portugal
No data for Coimbra Portugal
No data for Funchal Portugal
No data for Setubal Portugal
looking for Qatar
looking for Romania
looking for Senegal
looking for Singapore
looking for El Salvador
looking for Serbia
looking for Slovakia
looking for Slovenia
looking for Sweden
No data for Göteborg Sweden
No data for Helsingborg Sweden
No data for Malmö Sweden
No data for Orebro Sweden
No data for Stockholm Sweden
No data for Uppsala Sweden
No data for Västerås Sweden
looking for Thailand
No data for Bangkok Thailand
No data for Chachoengsao Thailand
No data for Chiang Mai Thailand
No data for Chiang Rai Thailand
No data for Chon Buri Thailand
No data for Khon Kaen Thailand
No data for Lampang Thailand
No data for Lamphun Thailand
No data for Loei T

,country,city,year,PM2.5,PM10,NO2,population
1,Argentina,Buenos Aires,2015,10.26,27.87,NaN,12847328.0
2,Argentina,Buenos Aires,2016,NaN,26.88,15.35,13879707.0
3,Argentina,Buenos Aires,2017,NaN,25.40,19.57,14340951.0
4,Argentina,Buenos Aires,2018,NaN,24.40,16.60,14802195.0
5,Argentina,Buenos Aires,2019,NaN,25.50,18.25,15263440.0
...,...,...,...,...,...,...,...
5965,Slovenia,Maribor,2017,19.12,27.53,27.32,107841.0
5966,Slovenia,Maribor,2018,16.94,27.89,22.30,108665.0
5967,Slovenia,Maribor,2019,12.87,22.64,24.51,109496.0
6130,Ukraine,Kyiv,2018,22.82,37.84,NaN,2893215.0


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np

if train_df is None:
    train_df = pd.read_pickle( 'model_data_3663.p' )

train_df_city = train_df
train_df_city.fillna( 0, inplace=True )

train_city, test_city = train_test_split( train_df_city, test_size=0.2, random_state=1 )
train_city, val_city = train_test_split( train_city, test_size=0.25, random_state=1 )

X = train_city[[ 'population' ]]
y = train_city[[ 'PM2.5', 'PM10', 'NO2' ]]

model = LinearRegression( )

model.fit( X, y )

def get_error_city( df ):
    X, y = df[[ 'population' ]], df[[ 'PM2.5', 'PM10', 'NO2' ]]
    predictions = model.predict( X )

    mae = mean_absolute_error( y, predictions )
    mse = mean_squared_error( y, predictions )
    rmse = np.sqrt( mse )

    print( f'Mean Absolute Error (MAE): { mae }' )
    print( f'Mean Squared Error (MSE): { mse }' )
    print( f'Root Mean Squared Error (RMSE): {rmse}' )

print( 'Validation:' )
get_error_city( val_city )
print( '\nTest:' )
get_error_city( test_city )

Validation:
Mean Absolute Error (MAE): 14.696744417565457
Mean Squared Error (MSE): 638.8105866107036
Root Mean Squared Error (RMSE): 25.27470250291195

Test:
Mean Absolute Error (MAE): 15.455058204281935
Mean Squared Error (MSE): 675.238173851746
Root Mean Squared Error (RMSE): 25.985345367182365


In [60]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Lasso

train_df_country = pd.read_pickle( 'country_dataset.p' )
train_df_country.fillna( 0, inplace=True )

train_country, test_country = train_test_split( train_df_country, test_size=0.2, random_state=1 )
train_country, val_country = train_test_split( train_country, test_size=0.25, random_state=1 )

all_feature_columns = [ 'population', 'gdp', 'cement_co2', 'cement_co2_per_capita', 'co2', 'co2_including_luc', 'co2_per_capita', 'co2_per_gdp', 'coal_co2', 'energy_per_capita', 'energy_per_gdp', 'gas_co2', 'gas_co2_per_capita', 'oil_co2', 'oil_co2_per_capita', 'other_co2_per_capita', 'primary_energy_consumption' ]

results = [ ]

models = {
    RandomForestRegressor : {
        'criterion' : [ 'poisson', 'friedman_mse', 'squared_error', 'absolute_error' ],
        'max_depth' : { 'min_' : 2, 'max_' : 40, 'type' : 'int' },
        'min_samples_split' : { 'type' : 'float' },
        'min_samples_leaf' : { 'type' : 'float' },
        'max_features' : [ 'sqrt', 'log2' ],
        'random_state' : { 'min_' : 1, 'max_' : 42, 'type' : 'int' }
    },
    DecisionTreeRegressor : {
        'criterion' : [ 'squared_error', 'friedman_mse', 'absolute_error', 'poisson' ],
        'max_depth' : { 'min_' : 2, 'max_' : 40, 'type' : 'int' },
        'min_samples_split' : { 'type' : 'float' },
        'min_samples_leaf' : { 'type' : 'float' },
        'max_features' : [ 'sqrt', 'log2' ],
        'random_state' : { 'min_' : 1, 'max_' : 42, 'type' : 'int' }
    },
    LinearRegression : {
        'fit_intercept' : [ True, False ]
    },
    Lasso : {
        'alpha' : { 'min_' : 0, 'max_' : 1_000_000, 'type' : 'int' }
    }
}

for model in models:
    parameters = { }
    for param_name in models[ model ]:
        if type( models[ model ][ param_name ] ) == list:
            parameters[ param_name ] = np.random.choice( models[ model ][ param_name ], 1, replace=False )[ 0 ]
        elif type( models[ model ][ param_name ] ) == dict:
            if models[ model ][ param_name ][ 'type' ] == 'int':
                parameters[ param_name ] = np.random.randint( models[ model ][ param_name ][ 'min_' ], models[ model ][ param_name ][ 'max_' ] )
            else:
                parameters[ param_name ] = np.random.random( )
    
    new_model = model( )
    new_model.set_params( **parameters )
    
    for _ in range( 1000 ):
        selected_columns = np.random.choice( all_feature_columns, np.random.randint( 2, len( all_feature_columns ) ), replace=False )
        X_train = train_country[ selected_columns ]
        y_train = train_country[[ 'PM2.5', 'PM10', 'NO2' ]]

        new_model.fit( X_train, y_train )

        def get_error_country( df ):
            X, y = df[ selected_columns ], df[[ 'PM2.5', 'PM10', 'NO2' ]]

            predictions = new_model.predict( X )

            mae = mean_absolute_error( y, predictions )
            mse = mean_squared_error( y, predictions )
            rmse = np.sqrt( mse )

            return mae, rmse

        val_mae, val_rmse = get_error_country( val_country )
        test_mae, test_rmse = get_error_country( test_country )

        results.append({
            'val_mae'   : val_mae,
            'val_rmse'  : val_rmse,
            'test_mae'  : test_mae,
            'test_rmse' : test_rmse,
            'features'  : selected_columns,
            'model' : new_model
        })

results = sorted( results, key=lambda d: ( d[ 'val_rmse' ], d[ 'test_rmse' ] ) )

for i in range( 5 ):
    print( f'#{ i + 1 } best model was { results[ i ][ "model" ] } with rmse of { results[ i ][ "val_rmse" ] }/{ results[ i ][ "test_rmse" ] } (features({ len( results[ i ][ "features" ] ) }): { ", ".join( results[ i ][ "features" ] ) }) \n\n' )
    
    

#1 best model was LinearRegression(fit_intercept=False) with rmse of 27.98582897616052/28.92174462964388 (features(12): cement_co2_per_capita, oil_co2, gas_co2_per_capita, coal_co2, co2, gas_co2, co2_including_luc, co2_per_capita, oil_co2_per_capita, other_co2_per_capita, population, gdp) 


#2 best model was LinearRegression(fit_intercept=False) with rmse of 28.157897609677168/28.69435501702439 (features(13): cement_co2, cement_co2_per_capita, other_co2_per_capita, gas_co2_per_capita, co2_including_luc, oil_co2, co2, co2_per_capita, population, primary_energy_consumption, gas_co2, oil_co2_per_capita, coal_co2) 


#3 best model was LinearRegression(fit_intercept=False) with rmse of 28.192495058656696/27.785504108918314 (features(15): oil_co2_per_capita, oil_co2, gas_co2_per_capita, co2_including_luc, gas_co2, gdp, primary_energy_consumption, cement_co2, population, energy_per_gdp, other_co2_per_capita, co2, cement_co2_per_capita, coal_co2, co2_per_capita) 


#4 best model was LinearReg